In [1]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import lightgbm as lgb
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn

from ultimate.mlp import MLP 

from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [4]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)


/home/zac_farnsworth95/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
print(len(trainset))

4446965


# Normalization and feature engineering

Add a feature for the number of players joined

Either drop the matchtype or one-hot encode it. Dropping it works better.

In [12]:
# partially inspired by https://www.kaggle.com/modmari/deep-neural-network-predicting-pubg-winners
def feature_engineering():
    df = pd.read_csv("pubg-finish-placement-prediction/train_V2_clean.csv",index_col=0)
    df = df[df['maxPlace'] > 1]
 
    df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]
    
    df['rankPoints'] = np.where(df['rankPoints'] <= 0 ,0 , df['rankPoints'])
    
    df['playersJoined'] = df.groupby('matchId')['matchId'].transform('count')
    df['killsNorm'] = df['kills']*((100-df['playersJoined'])/100 + 1)
    df['damageDealtNorm'] = df['damageDealt']*((100-df['playersJoined'])/100 + 1)
    df['maxPlaceNorm'] = df['maxPlace']*((100-df['playersJoined'])/100 + 1)
    df['matchDurationNorm'] = df['matchDuration']*((100-df['playersJoined'])/100 + 1)
    df['healsandboosts'] = df['heals'] + df['boosts']
    df['totalDistance'] = df['rideDistance'] + df['walkDistance'] + df['swimDistance']
    df['killsWithoutMoving'] = ((df['kills'] > 0) & (df['totalDistance'] == 0))
    df['headshot_rate'] = df['headshotKills'] / df['kills']
    df['headshot_rate'] = df['headshot_rate'].fillna(0)
    df.drop(df[df['killsWithoutMoving'] == True].index, inplace=True)
    df.drop(df[df['roadKills'] > 8].index, inplace=True)
                           
    

    target = 'winPlacePerc'
    # Get a list of the features to be used
    features = list(df.columns)
    
    # Remove some features from the features list :
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchDuration")
    features.remove("matchType")
    
    y = None
    
    y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
    # Remove the target from the features list :
    features.remove(target)
    
    # Make new features indicating the mean of the features ( grouped by match and group ) :
    print("get group mean feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    
    df_out = agg.reset_index()[['matchId','groupId']]
    
    # Merge agg and agg_rank (that we got before) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the max value of the features for each group ( grouped by match )
    print("get group max feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('max')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the minimum value of the features for each group ( grouped by match )
    print("get group min feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('min')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the number of players in each group ( grouped by match )
    print("get group size feature")
    agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
     
    # Merge the group_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the mean value of each features for each match :
    print("get match mean feature")
    agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
    
    # Merge the new agg with df_out :
    df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
    
    # Make new features indicating the number of groups in each match :
    print("get match size feature")
    agg = df.groupby(['matchId']).size().reset_index(name='match_size')
    
    # Merge the match_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId'])
    
    # Drop matchId and groupId
    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)
    
    y = y.tolist()
    

    return df_out,y

# Train on XGBoost

In [13]:
x,y = feature_engineering()



get group mean feature
get group max feature
get group min feature
get group size feature
get match mean feature
get match size feature


In [14]:
#x = pd.get_dummies(x, columns=['matchType'])


In [15]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [16]:
x_train.head()

,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,killPlace_mean,killPoints_mean,kills_mean,killStreaks_mean,...,totalDistance,playersJoined,killsNorm,damageDealtNorm,maxPlaceNorm,matchDurationNorm,healsandboosts,killsWithoutMoving,headshot_rate,match_size
1359366,0.0,0.0,29.37,0.0,0.0,0.0,78.5,0.0,0.0,0.0,...,2539.780457,92.0,0.997826,139.334439,50.76,2021.76,3.445652,False,0.103986,92
425492,0.0,2.0,154.85,1.5,1.0,1.0,20.0,1071.5,2.0,1.5,...,1611.002446,91.0,0.994176,152.342772,29.43,1486.76,2.769231,False,0.115145,91
1368208,0.5,3.0,320.25,1.0,1.0,5.0,14.5,1255.5,2.5,1.0,...,1917.370526,95.0,0.961579,149.348242,50.40,1945.65,2.421053,False,0.092982,95
437680,0.0,0.0,0.00,0.0,0.0,0.0,88.0,0.0,0.0,0.0,...,1001.585637,92.0,1.044783,127.115883,99.36,1461.24,1.967391,False,0.101268,92
612460,1.0,1.0,510.50,2.0,1.0,6.0,7.0,0.0,3.0,1.0,...,1319.039106,93.0,0.966452,134.852905,29.96,1527.96,2.032258,False,0.102509,93


In [17]:
trainset_dmatrix = xgb.DMatrix(x_train.values,label=y_train,feature_names=x_train.columns)
valset_dmatrix = xgb.DMatrix(x_val.values,label=y_val,feature_names=x_val.columns)

In [18]:
params = {
    "max_depth" : 20,
    "eval_metric" : ["mae"],
#     "eta" : 0.1,
     "gamma" : 0.1,
}

In [19]:
clf = xgb.train(params, trainset_dmatrix, evals=[(trainset_dmatrix, "train"),(valset_dmatrix, 'val')], num_boost_round = 30)

predictions = xgb.predict(valset_dmatrix)

[22:30:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8954 extra nodes, 94928 pruned nodes, max_depth=20
[0]	train-mae:0.183134	val-mae:0.182955
[22:30:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 7870 extra nodes, 120968 pruned nodes, max_depth=20
[1]	train-mae:0.130768	val-mae:0.131041
[22:31:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6726 extra nodes, 147150 pruned nodes, max_depth=20
[2]	train-mae:0.095036	val-mae:0.095779
[22:32:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 5404 extra nodes, 169836 pruned nodes, max_depth=20
[3]	train-mae:0.071203	val-mae:0.072355
[22:32:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4202 extra nodes, 181938 pruned nodes, max_depth=20
[4]	train-mae:0.055702	val-mae:0.05721
[22:33:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3266 extra nodes, 191058 pruned nodes, max_depth=20
[5]	train-mae:0.

AttributeError: module 'xgboost' has no attribute 'predict'

In [ ]:
besttrain = .0280
bestval = .0305

# LGB Stuff

In [13]:
d_train = lgb.Dataset(x_train, label=y_train)
d_eval = lgb.Dataset(x_val, label=y_val, reference=d_train)

In [14]:
# params = {
#     "metric" : ["mae"],
#     "learning_rate" : 0.02,
#     'bagging_fraction': 0.7,
#     'bagging_freq': 1,
#     'feature_fraction': 0.7,
#     'num_leaves': 2047,
#     'min_data_in_leaf' : 100,
    
# }

params = {
    'boosting_type':'gbdt',
    'learning_rate': 0.1, 
    'max_depth': -1,
    'num_leaves': 30,
    'feature_fraction': 0.9,
    'subsample': 0.8,
    'min_data_in_leaf': 100,
    'lambda_l2': 4,
    'objective': 'regression_l2', 
    'zero_as_missing': True,
    'metric': 'mae',
    'seed': 2
}

In [15]:
gbm = lgb.train(params, d_train, valid_sets=[d_train,d_eval], num_boost_round=20000)

[1]	training's l1: 0.234342	valid_1's l1: 0.233843
[2]	training's l1: 0.212508	valid_1's l1: 0.212055
[3]	training's l1: 0.193005	valid_1's l1: 0.192606
[4]	training's l1: 0.17555	valid_1's l1: 0.175195
[5]	training's l1: 0.160079	valid_1's l1: 0.159738
[6]	training's l1: 0.146083	valid_1's l1: 0.145773
[7]	training's l1: 0.133722	valid_1's l1: 0.133437
[8]	training's l1: 0.122603	valid_1's l1: 0.12235
[9]	training's l1: 0.112712	valid_1's l1: 0.11249
[10]	training's l1: 0.103739	valid_1's l1: 0.10354
[11]	training's l1: 0.0958425	valid_1's l1: 0.0956608
[12]	training's l1: 0.0889034	valid_1's l1: 0.0887467
[13]	training's l1: 0.0827392	valid_1's l1: 0.0826189
[14]	training's l1: 0.0775146	valid_1's l1: 0.0774194
[15]	training's l1: 0.0727027	valid_1's l1: 0.0726314
[16]	training's l1: 0.0684815	valid_1's l1: 0.0684277
[17]	training's l1: 0.0646902	valid_1's l1: 0.064644
[18]	training's l1: 0.0613196	valid_1's l1: 0.0612764
[19]	training's l1: 0.0584317	valid_1's l1: 0.0583979
[20]	tra

[153]	training's l1: 0.0318074	valid_1's l1: 0.0318724
[154]	training's l1: 0.0317886	valid_1's l1: 0.0318548
[155]	training's l1: 0.0317766	valid_1's l1: 0.0318433
[156]	training's l1: 0.0317607	valid_1's l1: 0.0318281
[157]	training's l1: 0.0317439	valid_1's l1: 0.031812
[158]	training's l1: 0.0317284	valid_1's l1: 0.0317963
[159]	training's l1: 0.0317151	valid_1's l1: 0.0317826
[160]	training's l1: 0.0317013	valid_1's l1: 0.031769
[161]	training's l1: 0.0316874	valid_1's l1: 0.0317573
[162]	training's l1: 0.0316714	valid_1's l1: 0.0317409
[163]	training's l1: 0.0316525	valid_1's l1: 0.0317224
[164]	training's l1: 0.0316335	valid_1's l1: 0.0317033
[165]	training's l1: 0.0316156	valid_1's l1: 0.0316859
[166]	training's l1: 0.0316016	valid_1's l1: 0.0316721
[167]	training's l1: 0.0315878	valid_1's l1: 0.031659
[168]	training's l1: 0.031569	valid_1's l1: 0.0316386
[169]	training's l1: 0.031553	valid_1's l1: 0.0316233
[170]	training's l1: 0.0315426	valid_1's l1: 0.031614
[171]	training's

[303]	training's l1: 0.0304137	valid_1's l1: 0.0305446
[304]	training's l1: 0.0304104	valid_1's l1: 0.0305414
[305]	training's l1: 0.0304078	valid_1's l1: 0.0305393
[306]	training's l1: 0.0304045	valid_1's l1: 0.0305361
[307]	training's l1: 0.0303981	valid_1's l1: 0.0305291
[308]	training's l1: 0.0303935	valid_1's l1: 0.030525
[309]	training's l1: 0.0303862	valid_1's l1: 0.0305184
[310]	training's l1: 0.0303811	valid_1's l1: 0.0305147
[311]	training's l1: 0.0303759	valid_1's l1: 0.03051
[312]	training's l1: 0.0303718	valid_1's l1: 0.0305069
[313]	training's l1: 0.0303649	valid_1's l1: 0.030501
[314]	training's l1: 0.0303578	valid_1's l1: 0.0304954
[315]	training's l1: 0.0303548	valid_1's l1: 0.0304919
[316]	training's l1: 0.030345	valid_1's l1: 0.0304822
[317]	training's l1: 0.0303413	valid_1's l1: 0.0304787
[318]	training's l1: 0.0303374	valid_1's l1: 0.0304748
[319]	training's l1: 0.0303336	valid_1's l1: 0.0304718
[320]	training's l1: 0.0303243	valid_1's l1: 0.0304635
[321]	training'

[453]	training's l1: 0.0298614	valid_1's l1: 0.0300771
[454]	training's l1: 0.0298577	valid_1's l1: 0.0300739
[455]	training's l1: 0.0298552	valid_1's l1: 0.0300712
[456]	training's l1: 0.029852	valid_1's l1: 0.030068
[457]	training's l1: 0.0298503	valid_1's l1: 0.0300677
[458]	training's l1: 0.0298476	valid_1's l1: 0.0300653
[459]	training's l1: 0.0298462	valid_1's l1: 0.0300646
[460]	training's l1: 0.0298419	valid_1's l1: 0.0300606
[461]	training's l1: 0.0298374	valid_1's l1: 0.0300558
[462]	training's l1: 0.0298348	valid_1's l1: 0.0300536
[463]	training's l1: 0.0298308	valid_1's l1: 0.0300505
[464]	training's l1: 0.0298297	valid_1's l1: 0.0300496
[465]	training's l1: 0.0298243	valid_1's l1: 0.0300444
[466]	training's l1: 0.0298228	valid_1's l1: 0.0300437
[467]	training's l1: 0.0298215	valid_1's l1: 0.0300431
[468]	training's l1: 0.0298196	valid_1's l1: 0.0300418
[469]	training's l1: 0.0298158	valid_1's l1: 0.0300388
[470]	training's l1: 0.0298151	valid_1's l1: 0.0300389
[471]	traini

[603]	training's l1: 0.0294665	valid_1's l1: 0.0297669
[604]	training's l1: 0.0294656	valid_1's l1: 0.0297668
[605]	training's l1: 0.0294643	valid_1's l1: 0.0297658
[606]	training's l1: 0.0294632	valid_1's l1: 0.0297655
[607]	training's l1: 0.0294622	valid_1's l1: 0.0297651
[608]	training's l1: 0.0294597	valid_1's l1: 0.0297626
[609]	training's l1: 0.0294555	valid_1's l1: 0.0297593
[610]	training's l1: 0.0294546	valid_1's l1: 0.0297593
[611]	training's l1: 0.0294522	valid_1's l1: 0.0297572
[612]	training's l1: 0.0294512	valid_1's l1: 0.0297563
[613]	training's l1: 0.0294502	valid_1's l1: 0.0297561
[614]	training's l1: 0.0294492	valid_1's l1: 0.0297562
[615]	training's l1: 0.0294464	valid_1's l1: 0.0297552
[616]	training's l1: 0.0294454	valid_1's l1: 0.0297548
[617]	training's l1: 0.0294417	valid_1's l1: 0.0297522
[618]	training's l1: 0.0294376	valid_1's l1: 0.0297489
[619]	training's l1: 0.0294355	valid_1's l1: 0.0297468
[620]	training's l1: 0.0294339	valid_1's l1: 0.0297455
[621]	trai

...

[19326]	training's l1: 0.0187074	valid_1's l1: 0.0261611
[19327]	training's l1: 0.018707	valid_1's l1: 0.0261612
[19328]	training's l1: 0.0187065	valid_1's l1: 0.0261609
[19329]	training's l1: 0.0187061	valid_1's l1: 0.0261609
[19330]	training's l1: 0.0187059	valid_1's l1: 0.0261608
[19331]	training's l1: 0.0187056	valid_1's l1: 0.0261608
[19332]	training's l1: 0.0187054	valid_1's l1: 0.0261607
[19333]	training's l1: 0.018705	valid_1's l1: 0.0261606
[19334]	training's l1: 0.0187047	valid_1's l1: 0.0261607
[19335]	training's l1: 0.0187045	valid_1's l1: 0.0261607
[19336]	training's l1: 0.0187041	valid_1's l1: 0.0261608
[19337]	training's l1: 0.0187039	valid_1's l1: 0.026161
[19338]	training's l1: 0.0187034	valid_1's l1: 0.0261609
[19339]	training's l1: 0.0187028	valid_1's l1: 0.0261605
[19340]	training's l1: 0.0187026	valid_1's l1: 0.0261604
[19341]	training's l1: 0.0187021	valid_1's l1: 0.02616
[19342]	training's l1: 0.0187018	valid_1's l1: 0.02616
[19343]	training's l1: 0.0187013	valid

[19472]	training's l1: 0.0186588	valid_1's l1: 0.0261496
[19473]	training's l1: 0.0186584	valid_1's l1: 0.0261495
[19474]	training's l1: 0.0186582	valid_1's l1: 0.0261495
[19475]	training's l1: 0.0186579	valid_1's l1: 0.0261496
[19476]	training's l1: 0.0186575	valid_1's l1: 0.0261492
[19477]	training's l1: 0.0186571	valid_1's l1: 0.026149
[19478]	training's l1: 0.0186568	valid_1's l1: 0.026149
[19479]	training's l1: 0.0186566	valid_1's l1: 0.0261491
[19480]	training's l1: 0.0186564	valid_1's l1: 0.0261491
[19481]	training's l1: 0.018656	valid_1's l1: 0.026149
[19482]	training's l1: 0.0186557	valid_1's l1: 0.026149
[19483]	training's l1: 0.0186553	valid_1's l1: 0.0261489
[19484]	training's l1: 0.0186551	valid_1's l1: 0.0261487
[19485]	training's l1: 0.0186548	valid_1's l1: 0.0261486
[19486]	training's l1: 0.0186545	valid_1's l1: 0.0261484
[19487]	training's l1: 0.0186543	valid_1's l1: 0.0261485
[19488]	training's l1: 0.0186541	valid_1's l1: 0.0261485
[19489]	training's l1: 0.0186539	val

[19618]	training's l1: 0.01861	valid_1's l1: 0.0261414
[19619]	training's l1: 0.0186098	valid_1's l1: 0.0261413
[19620]	training's l1: 0.0186094	valid_1's l1: 0.0261412
[19621]	training's l1: 0.018609	valid_1's l1: 0.0261411
[19622]	training's l1: 0.0186085	valid_1's l1: 0.0261407
[19623]	training's l1: 0.0186082	valid_1's l1: 0.0261409
[19624]	training's l1: 0.0186079	valid_1's l1: 0.026141
[19625]	training's l1: 0.0186076	valid_1's l1: 0.026141
[19626]	training's l1: 0.0186074	valid_1's l1: 0.026141
[19627]	training's l1: 0.0186072	valid_1's l1: 0.0261409
[19628]	training's l1: 0.0186069	valid_1's l1: 0.0261409
[19629]	training's l1: 0.0186066	valid_1's l1: 0.0261412
[19630]	training's l1: 0.0186064	valid_1's l1: 0.026141
[19631]	training's l1: 0.018606	valid_1's l1: 0.026141
[19632]	training's l1: 0.0186056	valid_1's l1: 0.0261408
[19633]	training's l1: 0.0186053	valid_1's l1: 0.0261411
[19634]	training's l1: 0.018605	valid_1's l1: 0.0261414
[19635]	training's l1: 0.0186046	valid_1'

[19763]	training's l1: 0.0185621	valid_1's l1: 0.0261329
[19764]	training's l1: 0.0185619	valid_1's l1: 0.0261328
[19765]	training's l1: 0.0185615	valid_1's l1: 0.0261323
[19766]	training's l1: 0.0185613	valid_1's l1: 0.0261323
[19767]	training's l1: 0.018561	valid_1's l1: 0.0261323
[19768]	training's l1: 0.0185607	valid_1's l1: 0.0261325
[19769]	training's l1: 0.0185604	valid_1's l1: 0.0261324
[19770]	training's l1: 0.01856	valid_1's l1: 0.0261324
[19771]	training's l1: 0.0185595	valid_1's l1: 0.0261323
[19772]	training's l1: 0.0185584	valid_1's l1: 0.0261315
[19773]	training's l1: 0.0185579	valid_1's l1: 0.0261311
[19774]	training's l1: 0.0185576	valid_1's l1: 0.0261309
[19775]	training's l1: 0.0185573	valid_1's l1: 0.0261309
[19776]	training's l1: 0.0185568	valid_1's l1: 0.0261307
[19777]	training's l1: 0.0185565	valid_1's l1: 0.0261305
[19778]	training's l1: 0.0185562	valid_1's l1: 0.0261305
[19779]	training's l1: 0.0185559	valid_1's l1: 0.0261303
[19780]	training's l1: 0.0185556	v

[19908]	training's l1: 0.0185136	valid_1's l1: 0.0261259
[19909]	training's l1: 0.0185134	valid_1's l1: 0.0261259
[19910]	training's l1: 0.018513	valid_1's l1: 0.0261257
[19911]	training's l1: 0.0185127	valid_1's l1: 0.0261256
[19912]	training's l1: 0.0185123	valid_1's l1: 0.0261254
[19913]	training's l1: 0.0185115	valid_1's l1: 0.0261249
[19914]	training's l1: 0.0185112	valid_1's l1: 0.0261247
[19915]	training's l1: 0.0185105	valid_1's l1: 0.0261243
[19916]	training's l1: 0.0185101	valid_1's l1: 0.026124
[19917]	training's l1: 0.0185097	valid_1's l1: 0.0261242
[19918]	training's l1: 0.0185094	valid_1's l1: 0.026124
[19919]	training's l1: 0.018509	valid_1's l1: 0.0261238
[19920]	training's l1: 0.0185088	valid_1's l1: 0.026124
[19921]	training's l1: 0.0185084	valid_1's l1: 0.0261237
[19922]	training's l1: 0.0185081	valid_1's l1: 0.0261236
[19923]	training's l1: 0.0185079	valid_1's l1: 0.0261237
[19924]	training's l1: 0.0185074	valid_1's l1: 0.0261236
[19925]	training's l1: 0.0185071	val

In [ ]:
leaves = 2047
iterations = 5400
besttrain = .007108
bestval = .02419

# NN

In [20]:
# create NN_model
NN_model = Sequential()
NN_model.add(Dense(x.shape[1],  input_dim = x.shape[1], activation='relu'))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(BatchNormalization())
NN_model.add(Dropout(rate=0.2))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(BatchNormalization())
NN_model.add(Dropout(rate=0.2))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(BatchNormalization())
NN_model.add(Dropout(rate=0.2))
NN_model.add(Dense(100, activation='relu'))





# output Layer
NN_model.add(Dense(1, activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 226)               51302     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               22700     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [21]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [22]:
NN_model.fit(x=x, y=y, batch_size=1000,
             epochs=20, verbose=1, callbacks=callbacks_list,
             validation_split=0.2, validation_data=None, shuffle=True,
             class_weight=None, sample_weight=None, initial_epoch=0,
             steps_per_epoch=None, validation_steps=None)

Train on 1621177 samples, validate on 405295 samples
Epoch 1/20
1621177/1621177 [==============================] - 263s 162us/step - loss: 0.0927 - mean_absolute_error: 0.0927 - val_loss: 0.0636 - val_mean_absolute_error: 0.0636

Epoch 00001: val_loss improved from inf to 0.06364, saving model to Weights-001--0.06364.hdf5
Epoch 2/20
1621177/1621177 [==============================] - 258s 159us/step - loss: 0.0617 - mean_absolute_error: 0.0617 - val_loss: 0.0614 - val_mean_absolute_error: 0.0614

Epoch 00002: val_loss improved from 0.06364 to 0.06137, saving model to Weights-002--0.06137.hdf5
Epoch 3/20
1621177/1621177 [==============================] - 260s 161us/step - loss: 0.0550 - mean_absolute_error: 0.0550 - val_loss: 0.0545 - val_mean_absolute_error: 0.0545

Epoch 00003: val_loss improved from 0.06137 to 0.05454, saving model to Weights-003--0.05454.hdf5
Epoch 4/20
1621177/1621177 [==============================] - 256s 158us/step - loss: 0.0509 - mean_absolute_error: 0.0509 - v